In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/Lightning/notebooks/model'

/content/drive/MyDrive/Lightning/notebooks/model


### Importing essential libraries

In [ ]:
from config import CONFIG
import xarray as xr
import numpy as np
from sklearn.preprocessing import StandardScaler
from Lightning_Binary_Classifier import LSTM_Conv2D_KDD
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras import callbacks

In [ ]:
channels = CONFIG["FEATURES_NAMES"]
ds_path = CONFIG["COMBINED_PATH"]
step = 6

### Creating method to structure meteorological data with 6 hour step time

In [ ]:
def preprocess_data(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		end_ix = i + n_steps
		if end_ix > len(sequence)-1 or end_ix+6 > len(sequence)-1:
			break
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:end_ix+6]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

### Creating method to structure lightning data with 3 hour step time

In [ ]:
def preprocess_data_2(sequence, n_steps):
	X = list()
	for i in range(len(sequence)):
		end_ix = i + n_steps
		if end_ix > len(sequence)-1 or end_ix+6 > len(sequence)-1:
			break
		seq_x = sequence[i:i+3]
		X.append(seq_x)
	return np.array(X)

In [ ]:
train_df = xr.open_dataset(ds_path).sel(time=slice("2015", "2018"))
val_df = xr.open_dataset(ds_path).sel(time=slice("2019", "2020"))

In [ ]:
dataset={}
norm={}
for i in channels:
  if i == 'r':
    temp=train_df[i].values[:,0,:,:]
  else:
    temp=train_df[i].values
  scalers = {}
  for j in range(temp.shape[1]):
    scalers[j] = StandardScaler()
    temp[:, j, :] = scalers[j].fit_transform(temp[:, j, :]) 
  dataset[i]=temp
  norm[i]=scalers
  print(i)

p80.162
p79.162
cape
r


In [ ]:
def create_input(channels, ds_path, n_steps):
	stack = []
	for i in channels:
		input, _ = preprocess_data(dataset[i], n_steps=step)
		print(f"Adding channel {i} with shape: {input.shape}")
		stack.append(input)
	out = np.stack(stack, axis=-1)
	return out

In [ ]:
train_input1 = create_input(channels, ds_path, step)


#Encoder-2 Input 
train_input2 = preprocess_data_2(train_df['Amplitude'].values, n_steps=step)
train_input2 = np.expand_dims(train_input2, axis=-1) 

temp,_ = preprocess_data(train_df['Amplitude'].values, n_steps=step)
train_input3 = temp[:,5,:,:]
train_input3 = np.expand_dims(train_input3, axis=1)

#Decoder Output
_, train_target = preprocess_data(train_df['Amplitude'].values, n_steps=step)
train_target = np.expand_dims(train_target, axis=-1)
train_target = np.reshape(train_target, (-1,6, 29 * 33, 1))

Adding channel p80.162 with shape: (5844, 6, 29, 33)
Adding channel p79.162 with shape: (5844, 6, 29, 33)
Adding channel cape with shape: (5844, 6, 29, 33)
Adding channel r with shape: (5844, 6, 29, 33)


### Viewing the training input shape.

In [ ]:
print(train_input1.shape)
print(train_input2.shape)
print(train_input3.shape)
print(train_target.shape)

(5844, 6, 29, 33, 4)
(5844, 3, 29, 33, 1)
(5844, 1, 29, 33)
(5844, 6, 957, 1)


In [ ]:
#Validation Data
for i in channels:
  if i == 'r':
    temp=val_df[i].values[:,0,:,:]
  else:  
    temp=val_df[i].values
  scalers = {}
  for j in range(temp.shape[1]):
    scalers[j] = StandardScaler()
    temp[:, j, :] = scalers[j].fit_transform(temp[:, j, :]) 
  dataset[i]=temp
  norm[i]=scalers
  print(i)

p80.162
p79.162
cape
r


In [ ]:
#Encoder-1 Input
def create_input(channels, ds_path, n_steps):
	stack = []
	for i in channels:
		input, _ = preprocess_data(dataset[i], n_steps=step)
		print(f"Adding channel {i} with shape: {input.shape}")
		stack.append(input)
	out = np.stack(stack, axis=-1)
	return out

In [ ]:
val_input1 = create_input(channels, ds_path, step)

#Encoder-2 Input 
val_input2 = preprocess_data_2(val_df['Amplitude'].values, n_steps=step)
val_input2 = np.expand_dims(val_input2, axis=-1) 

temp,_ = preprocess_data(val_df['Amplitude'].values, n_steps=step)
val_input3 = temp[:,5,:,:]
val_input3 = np.expand_dims(val_input3, axis=1)

#Decoder Output
_, val_target = preprocess_data(val_df['Amplitude'].values, n_steps=step)
val_target = np.expand_dims(val_target, axis=-1)
val_target = np.reshape(val_target, (-1,6, 29 * 33, 1))

Adding channel p80.162 with shape: (2916, 6, 29, 33)
Adding channel p79.162 with shape: (2916, 6, 29, 33)
Adding channel cape with shape: (2916, 6, 29, 33)
Adding channel r with shape: (2916, 6, 29, 33)


### Viewing the validation data shape.

In [ ]:
print(val_input1.shape)
print(val_input2.shape)
print(val_input3.shape)
print(val_target.shape)

(2916, 6, 29, 33, 4)
(2916, 3, 29, 33, 1)
(2916, 1, 29, 33)
(2916, 6, 957, 1)


In [ ]:
train_data_X = [train_input1 ,train_input2 ,train_input3]
val_data_X   = [val_input1 ,val_input2 ,val_input3]

train_data_Y = train_target
val_data_Y = val_target

model = LSTM_Conv2D_KDD()

### Summary of the model being used

In [21]:
model.summary()

Model: "ConvLSTM-Conv2d-KDD"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder1_inputs (InputLayer)   [(None, None, 29, 3  0           []                               
                                3, 4)]                                                            
                                                                                                  
 encoder2_inputs (InputLayer)   [(None, None, 29, 3  0           []                               
                                3, 1)]                                                            
                                                                                                  
 en1_conv2d (TimeDistributed)   (None, None, 15, 17  12608       ['encoder1_inputs[0][0]']        
                                , 64)                                           

### Defing the metrics

In [ ]:
#Metrics
def POD(y_true, y_pred):
    ytrue = y_true
    ypred = K.sigmoid(y_pred)
    ypred = K.round(ypred)
    true_positives = K.sum(ytrue * ypred)
    possible_positives = K.sum(ytrue)
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def FAR(y_true, y_pred):
    ytrue = y_true
    ypred = K.sigmoid(y_pred)
    ypred = K.round(ypred)
    true_positives = K.sum(ytrue * ypred)
    predicted_positives = K.sum(ypred)
    precision = true_positives / (predicted_positives + K.epsilon())
    return 1 - precision

def binary_acc(y_true,y_pred):
    ypred = K.sigmoid(y_pred)
    return K.mean(K.equal(y_true, K.round(ypred)), axis=-1)

### Defing the loss function

In [ ]:
def weight_loss(y_true,y_pred):  # binary classification
    pw = 4
    ytrue = K.flatten(y_true)
    ypred = K.flatten(y_pred)
    return tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(logits=ypred,labels=ytrue,pos_weight=pw))

### Compiling the model

In [ ]:
adam = optimizers.Adam(lr=0.0001)
model.compile(loss = weight_loss,optimizer = adam, metrics=[POD,FAR,binary_acc])

logdir = "/content/drive/MyDrive/Lightning/logs"
call1 = callbacks.TensorBoard(log_dir=logdir, histogram_freq=10, write_graph=True)

modelfileDir="/content/drive/MyDrive/Lightning/saved models"
call2 = callbacks.ModelCheckpoint(modelfileDir, monitor='val_FAR', verbose=1, save_best_only=True, mode='min', save_weights_only=False)

call3 = callbacks.ReduceLROnPlateau()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


### Training the model

In [ ]:
model.fit(
x=train_data_X,
y=train_data_Y,
validation_data=(val_data_X,val_data_Y),
epochs=50,
batch_size=8,
shuffle=False,
callbacks=[call1,call2,call3],
)

Epoch 1/50
731/731 [==============================] - ETA: 0s - loss: 0.3996 - POD: 0.0038 - FAR: 0.9881 - binary_acc: 0.9655
Epoch 00001: val_FAR improved from inf to 0.93211, saving model to /content/drive/MyDrive/Lightning/saved models
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Lightning/saved models/assets
731/731 [==============================] - 136s 149ms/step - loss: 0.3996 - POD: 0.0038 - FAR: 0.9881 - binary_acc: 0.9655 - val_loss: 0.2726 - val_POD: 0.0116 - val_FAR: 0.9321 - val_binary_acc: 0.9827 - lr: 1.0000e-04
Epoch 2/50
731/731 [==============================] - ETA: 0s - loss: 0.3194 - POD: 0.0980 - FAR: 0.7921 - binary_acc: 0.9616
Epoch 00002: val_FAR improved from 0.93211 to 0.90648, saving model to /content/drive/MyDrive/Lightning/saved models
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Lightning/saved models/assets
731/731 [==============================] - 105s 144ms/step - loss: 0.3194 - POD: 0.0980 - FAR: 0.7921 - binary_acc: 0.9616